In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from random import randint

import re
import os
from datetime import datetime

%config Completer.use_jedi = False

### Collecting sale hrefs from pages with different search filters

In [5]:
driver = webdriver.Chrome()

In [6]:
# options for url generation

# whether flat is presented by agency or owner
agent = ['&is_by_homeowner=1', '']

# type of flat - new or secodary
object_type = ['&object_type%5B0%5D=2', '&object_type%5B0%5D=1']

# number of rooms
rooms = ['&room1=1', '&room2=1', '&room3=1', '&room4=1', '&room5=1', '&room6=1']

In [7]:
def create_url(agent, object_type, page, rooms):
    
    intro = 'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2'
    max_price = '&maxprice=50000000'
    offer_type = '&offer_type=flat'
    page = '&p=' + str(page)
    region = '&region=1'
    
    return intro + agent + max_price + object_type + offer_type + page + region + rooms

In [8]:
# create_url(agent[0], object_type[0], 1, rooms[0])

In [9]:
def get_page_hrefs(url):
    
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
#     checking if page if there are any ads which satisfy given filters
    if soup.find('h3', attrs={'data-name': 'EmptySearchContent'}) != None:
        return 'generate_new_url'
        
#     extracting all ads' hrefs from current page
    elements = soup.find_all('div', attrs={'data-name': 'LinkArea'})

    hrefs = [item.find('a')['href'] for item in elements]

    return hrefs

In [10]:
# get_page_hrefs(create_url(agent[0], object_type[0], 1, rooms[0]))

In [11]:
all_hrefs = []

# iterating through parameters for creating filters in url
for ag in agent:
    for obj_t in object_type:
        for r in rooms:
            
#             getting hrefs for a particular combination
            page = 1
            
#             55 - is the maximum number of pages, that CIAN gives for each search request
            while page != 55:
                
                url = create_url(ag, obj_t, page, r)
                hrefs = get_page_hrefs(url) 
                
#                 if such filters combination returns empty page, cycle will switch to the next for iteration
                if hrefs == 'generate_new_url':
                    
                    time.sleep(randint(1, 3))
                    print('=== url with given filters returns no matches ===')
                    break
                
#                 when pages come to an end, CIAN doesn't raise an error
#                 instead, they redirect you to the first page
#                 to recognise such behavior, we'll save our first page href 

                if page == 1:
                    
                    first_url = hrefs[0]
                
#                 on each while iteration we'll compare first href on page with our saved one from above
#                 if they match, we know that redirection happend and while loop can be broken
#                 moving on to the next for loop with new filter combinations

                if hrefs[0] == first_url and page != 1:
                    
                    time.sleep(randint(1, 3))
                    print(' === run out of pages ===')
                    break
                   
                
#                 adding one page hrefs to a storage list
                all_hrefs.extend(hrefs)
                print(f'collected {page} page(s)')
    
                page += 1
                time.sleep(randint(1, 3))

=== url with given filters returns no matches ===
=== url with given filters returns no matches ===
=== url with given filters returns no matches ===
=== url with given filters returns no matches ===
=== url with given filters returns no matches ===
=== url with given filters returns no matches ===
collected 1 page(s)
collected 2 page(s)
collected 3 page(s)
collected 4 page(s)
collected 5 page(s)
collected 6 page(s)
collected 7 page(s)
collected 8 page(s)
collected 9 page(s)
collected 10 page(s)
collected 11 page(s)
collected 12 page(s)
collected 13 page(s)
collected 14 page(s)
collected 15 page(s)
collected 16 page(s)
collected 17 page(s)
collected 18 page(s)
collected 19 page(s)
collected 20 page(s)
collected 21 page(s)
collected 22 page(s)
 === run out of pages ===
collected 1 page(s)
collected 2 page(s)
collected 3 page(s)
collected 4 page(s)
collected 5 page(s)
collected 6 page(s)
collected 7 page(s)
collected 8 page(s)
collected 9 page(s)
collected 10 page(s)
collected 11 page(s)

collected 1 page(s)
collected 2 page(s)
collected 3 page(s)
collected 4 page(s)
collected 5 page(s)
collected 6 page(s)
collected 7 page(s)
collected 8 page(s)
collected 9 page(s)
collected 10 page(s)
collected 11 page(s)
collected 12 page(s)
collected 13 page(s)
collected 14 page(s)
collected 15 page(s)
collected 16 page(s)
collected 17 page(s)
collected 18 page(s)
collected 19 page(s)
collected 20 page(s)
collected 21 page(s)
collected 22 page(s)
collected 23 page(s)
collected 24 page(s)
collected 25 page(s)
collected 26 page(s)
collected 27 page(s)
collected 28 page(s)
collected 29 page(s)
collected 30 page(s)
collected 31 page(s)
collected 32 page(s)
collected 33 page(s)
collected 34 page(s)
collected 35 page(s)
collected 36 page(s)
collected 37 page(s)
collected 38 page(s)
collected 39 page(s)
collected 40 page(s)
collected 41 page(s)
collected 42 page(s)
collected 43 page(s)
collected 44 page(s)
collected 45 page(s)
collected 46 page(s)
collected 47 page(s)
collected 48 page(s)
c

In [15]:
with open('all_hrefs.txt', 'w') as file:
    file.writelines('\n'.join(all_hrefs))

In [12]:
len(all_hrefs)

13760

### Extracting offer and house info from 

In [12]:
with open('all_hrefs.txt') as file:
    all_hrefs = [line.strip() for line in file.readlines()]

In [171]:
soup = BeautifulSoup(requests.get(all_hrefs[2]).content, 'html.parser')

In [13]:
def get_price(s):
    
    try:
        price = s.find('span', attrs={'itemprop': 'price'})['content']
        price = ''.join(price.split()[:-1])
        return float(price)
    except:
        return None

In [173]:
extract_price(soup)

8950000.0

In [176]:
 main_data = soup.find_all('div', attrs={'class': 'a10a3f92e9--info--3XiXi'})

[<div class="a10a3f92e9--info--3XiXi"><div class="a10a3f92e9--info-value--18c8R">38,5 м²</div><div class="a10a3f92e9--info-title--2bXM9">Общая</div></div>,
 <div class="a10a3f92e9--info--3XiXi"><div class="a10a3f92e9--info-value--18c8R">19,1 м²</div><div class="a10a3f92e9--info-title--2bXM9">Жилая</div></div>,
 <div class="a10a3f92e9--info--3XiXi"><div class="a10a3f92e9--info-value--18c8R">8,2 м²</div><div class="a10a3f92e9--info-title--2bXM9">Кухня</div></div>,
 <div class="a10a3f92e9--info--3XiXi"><div class="a10a3f92e9--info-value--18c8R">5 из 17</div><div class="a10a3f92e9--info-title--2bXM9">Этаж</div></div>,
 <div class="a10a3f92e9--info--3XiXi"><div class="a10a3f92e9--info-value--18c8R">1988</div><div class="a10a3f92e9--info-title--2bXM9">Построен</div></div>]

In [14]:
def get_apartment_area(s):
    
    try:
        parent = s.find('div', text = 'Общая').parent
        child = parent.find(attrs={'class': 'a10a3f92e9--info-value--18c8R'}).text
        area = re.sub('\s', ' ', child).split(' ')[0].replace(',', '.')
        return float(area)
    except:
        return None

In [179]:
extract_apartment_area(soup)

38.5

In [15]:
def get_floor(s):
    
    try:
        parent = s.find('div', text = 'Этаж').parent
        child = parent.find(attrs={'class': 'a10a3f92e9--info-value--18c8R'}).text
        floors = list(map(int, child.split(' из ')))
        return floors
    except:
        return None

In [185]:
extract_floor(soup)

[5, 17]

In [16]:
def get_constr_year(s):
    
    try:
        parent = s.find('div', text = 'Построен').parent
        year = parent.find(attrs={'class': 'a10a3f92e9--info-value--18c8R'}).text
        return year
    except:
        return None

In [189]:
extract_constr_year(soup)

1988

In [17]:
def get_object_type(s):
    
    try:
        parent = s.find('span', text = 'Тип жилья').parent
        child = parent.find(attrs={'class': 'a10a3f92e9--value--3Ftu5'}).text
        return child
    except:
        return None

In [128]:
extract_object_type(soup)

'Вторичка'

In [18]:
def get_condition(s):
    
    try:
        parent = s.find('span', text = 'Ремонт').parent
        child = parent.find(attrs={'class': 'a10a3f92e9--value--3Ftu5'}).text
        return child
    except:
        return None

In [130]:
extract_condition(soup)

'Без ремонта'

In [19]:
def get_view(s):
    
    try:
        parent = s.find('span', text = 'Вид из окон').parent
        child = parent.find(attrs={'class': 'a10a3f92e9--value--3Ftu5'}).text
        return child
    except:
        return None

In [134]:
extract_view(soup)

'На улицу'

In [20]:
def get_parking(s):
    
    try:
        parent = s.find('div', text = 'Парковка').parent
        child = parent.find(attrs={'class': 'a10a3f92e9--value--38caj'}).text
        return child
    except:
        return None

In [146]:
extract_parking(soup)

'Подземная'

In [21]:
def get_closest_metro(s):
    access = s.find_all('li', attrs={'data-name': 'renderUnderground'})
    
    access_dct = {}
    for item in access:
        
        try:
            metro = item.find('a').text
            time = item.find('span').text
            access_dct[metro] = [int(re.search('[0-9]{1,2}', time).group(0)), re.search('на транспорте|пешком', time).group(0)]
        except:
            pass
        
    return access_dct

In [161]:
extract_closest_metro(soup)

{'Беговая': [2, 'на транспорте'],
 'Динамо': [9, 'пешком'],
 'Петровский Парк': [12, 'пешком']}

In [22]:
def get_header(s):
    
    try:
        header = s.find('h1', attrs={'class': 'a10a3f92e9--title--2Widg'}).text
        return header
    except:
        return None

In [166]:
get_header(soup)

'1-комн. квартира, 38,65 м²'

In [23]:
def get_district(s):
    
    try:
        district = s.find('address').find_all('a')[2].text
        return district
    except:
        return None

In [168]:
get_district(soup)

'р-н Беговой'

In [24]:
def get_mun_okr(s):
    
    try:
        mun_okr = s.find('address').find_all('a')[1].text
        return mun_okr
    except:
        return None

In [170]:
get_mun_okr(soup)

'САО'

In [25]:
name, mun_okr, district, metro, area, constr_year = [], [], [], [], [], []
obj_type, condition, view, parking, floor, price = [], [], [], [], [], []
href = []

save_suffix = 2

driver = webdriver.Chrome()

for idx, flat in zip(range(4731, len(all_hrefs) + 1), all_hrefs[4731:]):
    
    driver.get(flat)
    time.sleep(randint(1, 2))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
#     checking whether page was found, if wasn't - returning to the main page and moving to the next for iteration
    if soup.find('h1', text = 'Страница не найдена') != None:
        driver.find_element_by_xpath("//div[@class='button-container']/a").click()
        continue
    
#     reCAPTCHA check
#     we suppose that if three main received attributes are None, there is a reCAPTCHA to be solved
    while get_header(soup) == None and get_mun_okr(soup) == None and get_district(soup) == None and get_price(soup) == None:
        
        confirm_captcha = input()
        
        driver.get(flat)
        time.sleep(randint(1, 2))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
    
    name.append(get_header(soup))
    mun_okr.append(get_mun_okr(soup))
    district.append(get_district(soup))
    metro.append(get_closest_metro(soup))
    area.append(get_apartment_area(soup))
    constr_year.append(get_constr_year(soup))
    obj_type.append(get_object_type(soup))
    condition.append(get_condition(soup))
    view.append(get_view(soup))
    parking.append(get_parking(soup))
    floor.append(get_floor(soup))
    price.append(get_price(soup))
    href.append(flat)
    
    print(f'collected {idx + 1} flat(s)\t{datetime.now()}')
    
    with open('log.txt', 'a') as file:
        file.write(f'collected {idx + 1} flat(s)\t{datetime.now()}\n')
    
    if idx % 100 == 0 and idx != 0:
        
        df = pd.DataFrame([name, mun_okr, district, metro, area, constr_year,
                           obj_type, condition, view, parking, floor, price, href],
                          index = ['name', 'mun_distr', 'district', 'metro', 'area', 'constr_year',
                                   'obj_type', 'condition', 'view', 'parking', 'floor', 'price', 'link']).T
        
        df.to_pickle('flats_' + str(save_suffix) + '.pkl')
        save_suffix += 1
    
    time.sleep(randint(1, 5))

collected 2401 flat(s)	2021-02-10 22:55:46.158684
collected 2402 flat(s)	2021-02-10 22:55:57.631922
collected 2403 flat(s)	2021-02-10 22:56:07.544919
collected 2404 flat(s)	2021-02-10 22:56:15.844975
collected 2405 flat(s)	2021-02-10 22:56:23.720060
collected 2406 flat(s)	2021-02-10 22:56:34.336445
collected 2407 flat(s)	2021-02-10 22:56:49.176581
collected 2408 flat(s)	2021-02-10 22:56:59.636893
collected 2409 flat(s)	2021-02-10 22:57:11.237153
collected 2410 flat(s)	2021-02-10 22:57:21.004084
collected 2411 flat(s)	2021-02-10 22:57:30.821916
collected 2412 flat(s)	2021-02-10 22:57:41.473312
collected 2413 flat(s)	2021-02-10 22:57:50.323408
collected 2414 flat(s)	2021-02-10 22:57:58.426050
collected 2415 flat(s)	2021-02-10 22:58:08.402899
collected 2416 flat(s)	2021-02-10 22:58:18.341473
collected 2417 flat(s)	2021-02-10 22:58:28.310370
collected 2418 flat(s)	2021-02-10 22:58:36.624193
collected 2419 flat(s)	2021-02-10 22:58:45.440472
collected 2420 flat(s)	2021-02-10 22:58:52.890529


collected 2565 flat(s)	2021-02-10 23:20:59.417711
collected 2566 flat(s)	2021-02-10 23:21:05.810066
collected 2567 flat(s)	2021-02-10 23:21:16.406339
collected 2568 flat(s)	2021-02-10 23:21:25.350578
collected 2569 flat(s)	2021-02-10 23:21:36.054406
collected 2570 flat(s)	2021-02-10 23:21:43.979701
collected 2571 flat(s)	2021-02-10 23:21:50.972182
collected 2572 flat(s)	2021-02-10 23:21:59.801792
collected 2573 flat(s)	2021-02-10 23:22:07.377274
collected 2574 flat(s)	2021-02-10 23:22:15.428701
collected 2575 flat(s)	2021-02-10 23:22:23.563771
collected 2576 flat(s)	2021-02-10 23:22:34.860808
collected 2577 flat(s)	2021-02-10 23:22:46.193194
collected 2578 flat(s)	2021-02-10 23:22:56.262274
collected 2579 flat(s)	2021-02-10 23:23:07.091150
collected 2580 flat(s)	2021-02-10 23:23:16.529044
collected 2581 flat(s)	2021-02-10 23:23:25.229085
collected 2582 flat(s)	2021-02-10 23:23:35.687044
collected 2583 flat(s)	2021-02-10 23:23:45.003151
collected 2584 flat(s)	2021-02-10 23:23:56.776332


collected 2729 flat(s)	2021-02-10 23:46:21.167850
collected 2730 flat(s)	2021-02-10 23:46:30.906281
collected 2731 flat(s)	2021-02-10 23:46:40.377895
collected 2732 flat(s)	2021-02-10 23:46:48.589822
collected 2733 flat(s)	2021-02-10 23:46:57.009063
collected 2734 flat(s)	2021-02-10 23:47:08.420207
collected 2735 flat(s)	2021-02-10 23:47:14.574927
collected 2736 flat(s)	2021-02-10 23:47:23.337998
collected 2737 flat(s)	2021-02-10 23:47:30.361931
collected 2738 flat(s)	2021-02-10 23:47:37.779859
collected 2739 flat(s)	2021-02-10 23:47:48.233131
collected 2740 flat(s)	2021-02-10 23:47:55.499940
collected 2741 flat(s)	2021-02-10 23:48:06.634020
collected 2742 flat(s)	2021-02-10 23:48:16.203346
collected 2743 flat(s)	2021-02-10 23:48:27.588661
collected 2744 flat(s)	2021-02-10 23:48:36.112024
collected 2745 flat(s)	2021-02-10 23:48:43.852886
collected 2746 flat(s)	2021-02-10 23:48:51.816922
collected 2747 flat(s)	2021-02-10 23:49:01.241047
collected 2748 flat(s)	2021-02-10 23:49:11.825531


collected 2893 flat(s)	2021-02-11 00:11:37.052471
collected 2894 flat(s)	2021-02-11 00:11:49.814749
collected 2895 flat(s)	2021-02-11 00:12:01.649981
collected 2896 flat(s)	2021-02-11 00:12:11.039923
collected 2897 flat(s)	2021-02-11 00:12:21.153773
collected 2898 flat(s)	2021-02-11 00:12:31.001201
collected 2899 flat(s)	2021-02-11 00:12:41.260202
collected 2900 flat(s)	2021-02-11 00:12:52.878431
collected 2901 flat(s)	2021-02-11 00:13:02.382286
collected 2902 flat(s)	2021-02-11 00:13:13.671258
collected 2903 flat(s)	2021-02-11 00:13:23.914971
collected 2904 flat(s)	2021-02-11 00:13:33.165612
collected 2905 flat(s)	2021-02-11 00:13:41.357673
collected 2906 flat(s)	2021-02-11 00:13:49.295230
collected 2907 flat(s)	2021-02-11 00:13:58.016352
collected 2908 flat(s)	2021-02-11 00:14:10.250331
collected 2909 flat(s)	2021-02-11 00:14:22.048049
collected 2910 flat(s)	2021-02-11 00:14:30.294927
collected 2911 flat(s)	2021-02-11 00:14:41.744389
collected 2912 flat(s)	2021-02-11 00:14:52.049898


collected 3057 flat(s)	2021-02-11 00:37:59.776211
collected 3058 flat(s)	2021-02-11 00:38:09.688046
collected 3059 flat(s)	2021-02-11 00:38:19.467004
collected 3060 flat(s)	2021-02-11 00:38:27.076759
collected 3061 flat(s)	2021-02-11 00:38:36.924835
collected 3062 flat(s)	2021-02-11 00:38:46.637229
collected 3063 flat(s)	2021-02-11 00:38:58.256621
collected 3064 flat(s)	2021-02-11 00:39:08.501374
collected 3065 flat(s)	2021-02-11 00:39:14.308950
collected 3066 flat(s)	2021-02-11 00:39:20.986217
collected 3067 flat(s)	2021-02-11 00:39:30.894853
collected 3068 flat(s)	2021-02-11 00:39:38.651931
collected 3069 flat(s)	2021-02-11 00:39:49.767974
collected 3070 flat(s)	2021-02-11 00:39:56.767123
collected 3071 flat(s)	2021-02-11 00:40:03.033778
collected 3072 flat(s)	2021-02-11 00:40:13.696397
collected 3073 flat(s)	2021-02-11 00:40:22.621124
collected 3074 flat(s)	2021-02-11 00:40:32.100843
collected 3075 flat(s)	2021-02-11 00:40:42.811028
collected 3076 flat(s)	2021-02-11 00:40:53.990866


...

collected 4205 flat(s)	2021-02-11 04:03:46.063423
collected 4206 flat(s)	2021-02-11 04:03:57.443287
collected 4207 flat(s)	2021-02-11 04:04:05.501248
collected 4208 flat(s)	2021-02-11 04:04:14.211718
collected 4209 flat(s)	2021-02-11 04:04:20.000562
collected 4210 flat(s)	2021-02-11 04:04:27.927687
collected 4211 flat(s)	2021-02-11 04:04:33.725428
collected 4212 flat(s)	2021-02-11 04:04:41.459241
collected 4213 flat(s)	2021-02-11 04:04:51.635889
collected 4214 flat(s)	2021-02-11 04:04:59.731621
collected 4215 flat(s)	2021-02-11 04:05:08.826327
collected 4216 flat(s)	2021-02-11 04:05:17.813672
collected 4217 flat(s)	2021-02-11 04:06:05.971788
collected 4218 flat(s)	2021-02-11 04:06:13.857739
collected 4219 flat(s)	2021-02-11 04:06:21.737073
collected 4220 flat(s)	2021-02-11 04:06:33.000404
collected 4221 flat(s)	2021-02-11 04:06:44.415821
collected 4222 flat(s)	2021-02-11 04:06:51.990060
collected 4223 flat(s)	2021-02-11 04:06:58.480968
collected 4224 flat(s)	2021-02-11 04:07:08.099719


collected 4369 flat(s)	2021-02-11 04:33:53.529250
collected 4370 flat(s)	2021-02-11 04:34:03.376037
collected 4371 flat(s)	2021-02-11 04:34:11.328890
collected 4372 flat(s)	2021-02-11 04:34:21.995880
collected 4373 flat(s)	2021-02-11 04:34:29.682016
collected 4374 flat(s)	2021-02-11 04:34:38.763384
collected 4375 flat(s)	2021-02-11 04:34:48.606539
collected 4376 flat(s)	2021-02-11 04:34:55.300659
collected 4377 flat(s)	2021-02-11 04:35:06.802093
collected 4378 flat(s)	2021-02-11 04:35:17.155093
collected 4379 flat(s)	2021-02-11 04:35:26.051590
collected 4380 flat(s)	2021-02-11 04:35:34.653154
collected 4381 flat(s)	2021-02-11 04:35:43.632611
collected 4382 flat(s)	2021-02-11 04:35:52.420511
collected 4383 flat(s)	2021-02-11 04:36:04.391031
collected 4384 flat(s)	2021-02-11 04:36:19.997737
collected 4385 flat(s)	2021-02-11 04:36:31.436444
collected 4386 flat(s)	2021-02-11 04:36:41.384450
collected 4387 flat(s)	2021-02-11 04:36:55.566813
collected 4388 flat(s)	2021-02-11 04:37:11.205311


collected 4533 flat(s)	2021-02-11 05:03:08.094355
collected 4534 flat(s)	2021-02-11 05:03:19.159573
collected 4535 flat(s)	2021-02-11 05:03:30.568854
collected 4536 flat(s)	2021-02-11 05:03:42.314346
collected 4537 flat(s)	2021-02-11 05:03:50.628486
collected 4538 flat(s)	2021-02-11 05:04:01.512723
collected 4539 flat(s)	2021-02-11 05:04:11.199030
collected 4540 flat(s)	2021-02-11 05:04:18.020119
collected 4541 flat(s)	2021-02-11 05:04:28.107928
collected 4542 flat(s)	2021-02-11 05:04:34.730340
collected 4543 flat(s)	2021-02-11 05:04:45.108560
collected 4544 flat(s)	2021-02-11 05:04:54.050131
collected 4545 flat(s)	2021-02-11 05:05:04.005463
collected 4546 flat(s)	2021-02-11 05:05:17.623839
collected 4547 flat(s)	2021-02-11 05:05:26.071331
collected 4548 flat(s)	2021-02-11 05:05:40.553826
collected 4549 flat(s)	2021-02-11 05:06:01.483646
collected 4550 flat(s)	2021-02-11 05:06:14.266324
collected 4551 flat(s)	2021-02-11 05:06:29.637052
collected 4552 flat(s)	2021-02-11 05:06:46.910591


collected 4697 flat(s)	2021-02-11 05:32:57.283603
collected 4698 flat(s)	2021-02-11 05:33:13.771470
collected 4699 flat(s)	2021-02-11 05:33:21.234991
collected 4700 flat(s)	2021-02-11 05:33:30.305693
collected 4701 flat(s)	2021-02-11 05:33:37.059289
collected 4702 flat(s)	2021-02-11 05:33:44.770000
collected 4703 flat(s)	2021-02-11 05:33:56.636284
collected 4704 flat(s)	2021-02-11 05:34:06.091353
collected 4705 flat(s)	2021-02-11 05:34:17.595830
collected 4706 flat(s)	2021-02-11 05:34:28.290570
collected 4707 flat(s)	2021-02-11 05:34:36.829889
collected 4708 flat(s)	2021-02-11 05:34:45.935473
collected 4709 flat(s)	2021-02-11 05:34:57.063227
collected 4710 flat(s)	2021-02-11 05:35:06.845065
collected 4711 flat(s)	2021-02-11 05:35:14.316497
collected 4712 flat(s)	2021-02-11 05:35:23.909518
collected 4713 flat(s)	2021-02-11 05:36:00.866206
collected 4714 flat(s)	2021-02-11 05:36:12.589402
collected 4715 flat(s)	2021-02-11 05:36:21.571764
collected 4716 flat(s)	2021-02-11 05:36:28.391181


KeyboardInterrupt: Interrupted by user

In [28]:
df = pd.DataFrame([name, mun_okr, district, metro, area, constr_year,
                           obj_type, condition, view, parking, floor, price, href],
                          index = ['name', 'mun_distr', 'district', 'metro', 'area', 'constr_year',
                                   'obj_type', 'condition', 'view', 'parking', 'floor', 'price', 'link']).T
        
df.to_pickle('flats_' + str(save_suffix) + '.pkl')

In [29]:
pd.read_pickle('flats_26.pkl')

,name,mun_distr,district,metro,area,constr_year,obj_type,condition,view,parking,floor,price,link
0,"1-комн. квартира, 37 м²",СЗАО,р-н Хорошево-Мневники,"{'Хорошево': [20, 'пешком'], 'Полежаевская': [...",37.0,None,None,None,None,None,"[17, 23]",10900000.0,https://www.cian.ru/sale/flat/249632374/
1,"1-комн. апартаменты, 43,83 м²",ВАО,р-н Сокольники,"{'Сокольники': [20, 'пешком'], 'Рижская': [17,...",43.83,None,Новостройка Апартаменты,None,Во двор,None,"[11, 21]",10900000.0,https://www.cian.ru/sale/flat/250113902/
2,"1-комн. квартира, 45 м²",СВАО,р-н Марьина роща,"{'Бутырская': [4, 'на транспорте'], 'Дмитровск...",45.0,None,Новостройка,None,На улицу,None,"[7, 14]",11000000.0,https://www.cian.ru/sale/flat/247900381/
3,"1-комн. квартира, 50 м²",НАО (Новомосковский),Московский поселение,"{'Говорово': [10, 'пешком'], 'Тропарево': [5, ...",50.0,None,Новостройка,None,На улицу,None,"[2, 10]",11000000.0,https://www.cian.ru/sale/flat/239946252/
4,"1-комн. квартира, 38,4 м²",ЗАО,р-н Ново-Переделкино,"{'Новопеределкино': [9, 'пешком'], 'Боровское ...",38.4,None,Новостройка,None,None,None,"[12, 23]",11116838.0,https://www.cian.ru/sale/flat/247994173/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,"2-комн. квартира, 60,4 м²",ЮВАО,р-н Некрасовка,"{'Некрасовка': [18, 'пешком'], 'Лухмановская':...",60.4,None,Новостройка,None,None,None,"[10, 23]",9865695.0,https://www.cian.ru/sale/flat/250122673/
2328,"2-комн. квартира, 68,2 м²",ЮВАО,р-н Некрасовка,"{'Некрасовка': [13, 'пешком'], 'Лухмановская':...",68.2,None,Новостройка,None,None,None,"[9, 23]",9695477.0,https://www.cian.ru/sale/flat/250122782/
2329,"2-комн. квартира, 68,8 м²",НАО (Новомосковский),Московский,"{'Филатов Луг': [5, 'на транспорте'], 'Саларье...",68.8,None,Новостройка,None,None,None,"[14, 25]",9786389.0,https://www.cian.ru/sale/flat/250123469/
2330,"2-комн. апартаменты, 39,3 м²",САО,р-н Хорошевский,{},39.3,None,Новостройка Апартаменты,None,None,None,"[16, 22]",9821070.0,https://www.cian.ru/sale/flat/248462559/


In [251]:
all_hrefs[0]

'https://www.cian.ru/sale/flat/249687234/'

In [242]:
[file for file in os.listdir() if 'pkl' in file]

['flats_cian.pkl']

In [237]:
dataset = pd.DataFrame([name, mun_okr, district, metro, area, constr_year,
                        obj_type, condition, view, parking, floor, price],
                       index = ['name', 'mun_distr', 'district', 'metro', 'area', 'constr_year',
                                'obj_type', 'condition', 'view', 'parking', 'floor', 'price']).T

In [238]:
dataset

,name,mun_distr,district,metro,area,constr_year,obj_type,condition,view,parking,floor,price
0,"1-комн. квартира, 38,65 м²",САО,р-н Беговой,"{'Беговая': [2, 'на транспорте'], 'Динамо': [9...",38.65,2018,Вторичка,Без ремонта,На улицу,Подземная,"[13, 18]",15870000.0
1,"1-комн. квартира, 39 м²",САО,р-н Беговой,"{'Динамо': [7, 'пешком'], 'Петровский Парк': [...",39.0,2018,Вторичка,Без ремонта,NaN,NaN,"[14, 18]",15950000.0
2,"1-комн. квартира, 38,5 м²",ЮВАО,р-н Люблино,"{'Волжская': [6, 'пешком'], 'Кубанская': [14, ...",38.5,1988,Вторичка,Косметический,NaN,Наземная,"[5, 17]",8950000.0
3,"1-комн. квартира, 26 м²",ТАО (Троицкий),Краснопахорское поселение,"{'Прокшино': [17, 'на транспорте'], 'Саларьево...",26.0,NaN,NaN,NaN,NaN,NaN,"[3, 3]",2680000.0
4,"1-комн. квартира, 42 м²",НАО (Новомосковский),Сосенское поселение,"{'Прокшино': [15, 'пешком']}",42.0,2017,Вторичка,Дизайнерский,Во двор,NaN,"[14, 15]",9350000.0
5,"1-комн. квартира, 40 м²",СВАО,р-н Бутырский,"{'Дмитровская': [10, 'пешком'], 'Савеловская':...",40.0,2017,Вторичка,Евроремонт,NaN,Подземная,"[25, 47]",13700000.0
6,"1-комн. квартира, 32,5 м²",ЗАО,р-н Кунцево,"{'Кунцевская': [6, 'пешком'], 'Рабочий поселок...",32.5,1970,Вторичка,Косметический,Во двор,Наземная,"[3, 9]",8850000.0
7,"1-комн. квартира, 36,5 м²",СВАО,р-н Марфино,"{'Телецентр': [12, 'пешком'], 'Улица Милашенко...",36.5,2010,Вторичка,Евроремонт,NaN,NaN,"[5, 17]",12700000.0
8,"1-комн. квартира, 35 м²",САО,р-н Ховрино,"{'Ховрино': [10, 'пешком'], 'Беломорская': [10...",35.0,1965,Вторичка,Косметический,Во двор,NaN,"[11, 12]",8699999.0
9,"1-комн. квартира, 31,2 м²",ЮВАО,р-н Некрасовка,"{'Некрасовка': [24, 'пешком'], 'Лухмановская':...",31.2,NaN,Вторичка,Евроремонт,Во двор,NaN,"[7, 14]",7000000.0
